# Retrieve Route Attributes
We have a list of linkids, and we need to get the compile route attributes (how many turns, how much feet up, miles of bike facility, etc)

In [58]:
from pathlib import Path
import time
import geopandas as gpd
import numpy as np
import pickle

import modeling_turns
import stochastic_optimization

In [59]:
fp = Path.home() / 'Documents/BikewaySimData/Projects/gdot'

with (fp / 'impedance_calibration.pkl').open('rb') as fh:
    (df_edges,pseudo_df,pseudo_G) = pickle.load(fh)

In [61]:
source = list(pseudo_G.nodes())[0]
target = list(pseudo_G.nodes())[420]
print(source,target)

(783782570, 67358015) (67400350, 67400314)


In [62]:
import networkx as nx
length, path = nx.single_source_dijkstra(pseudo_G,source,target)

In [63]:
edge_list = [(path[i],path[i+1]) for i in range(len(path)-1)]
edge_list

[((783782570, 67358015), (67358015, 67377092)),
 ((67358015, 67377092), (67377092, 67433170)),
 ((67377092, 67433170), (67433170, 67379305)),
 ((67433170, 67379305), (67379305, 67379311)),
 ((67379305, 67379311), (67379311, 67379314)),
 ((67379311, 67379314), (67379314, 67408977)),
 ((67379314, 67408977), (67408977, 67410764)),
 ((67408977, 67410764), (67410764, 67376597)),
 ((67410764, 67376597), (67376597, 67376593)),
 ((67376597, 67376593), (67376593, 67375988)),
 ((67376593, 67375988), (67375988, 67375954)),
 ((67375988, 67375954), (67375954, 9666113491)),
 ((67375954, 9666113491), (9666113491, 67385397)),
 ((9666113491, 67385397), (67385397, 67385395)),
 ((67385397, 67385395), (67385395, 67466764)),
 ((67385395, 67466764), (67466764, 67456096)),
 ((67466764, 67456096), (67456096, 67384506)),
 ((67456096, 67384506), (67384506, 67384500)),
 ((67384506, 67384500), (67384500, 67384482)),
 ((67384500, 67384482), (67384482, 67473878)),
 ((67384482, 67473878), (67473878, 67440465)),
 ((6

In [18]:
# linkids = [32641.0, 33174.0, 33175.0, 35233.0, 35234.0, 35235.0, 32803.0, 36922.0, 36923.0, 32827.0, 36925.0, 32826.0, 33089.0, 33090.0, 32834.0, 32721.0, 32598.0, 33239.0, 33240.0, 33241.0, 32728.0, 32727.0, 32599.0, 35293.0, 35295.0, 32479.0, 32480.0, 32482.0, 32739.0, 32736.0, 32613.0, 32614.0, 32612.0, 32611.0, 32610.0, 32747.0, 32748.0, 32749.0, 33266.0, 33273.0, 33274.0, 33275.0, 33276.0, 33277.0]
# linkids = [int(x) for x in linkids]

In [20]:
# fp = Path.home() / 'Documents/BikewaySimData/Projects/gdot'

# #import network links
# nodes = gpd.read_file(fp/"networks/final_network.gpkg",layer='nodes')
# links = gpd.read_file(fp/"networks/final_network.gpkg",layer='links')#gpd.read_file(fp/"networks/final_network_w_elevation.gpkg")

# #TODO fix this
# links['oneway'] = links['oneway'] == "1"

# links_dict = links.set_index('linkid',drop=False,inplace=True)#.to_dict('index')

# # #%% prepare link dataframe
# # links['bike'] = links['bl'] + links['pbl'] + links['mu']
# # links['bike'] = links['bike'] >= 1

# links['length_ft'] = links.length

### Link Attributes

In [64]:
path

[(783782570, 67358015),
 (67358015, 67377092),
 (67377092, 67433170),
 (67433170, 67379305),
 (67379305, 67379311),
 (67379311, 67379314),
 (67379314, 67408977),
 (67408977, 67410764),
 (67410764, 67376597),
 (67376597, 67376593),
 (67376593, 67375988),
 (67375988, 67375954),
 (67375954, 9666113491),
 (9666113491, 67385397),
 (67385397, 67385395),
 (67385395, 67466764),
 (67466764, 67456096),
 (67456096, 67384506),
 (67384506, 67384500),
 (67384500, 67384482),
 (67384482, 67473878),
 (67473878, 67440465),
 (67440465, 67455439),
 (67455439, 67441191),
 (67441191, 67371161),
 (67371161, 5415065010),
 (5415065010, 5278817397),
 (5278817397, 67375776),
 (67375776, 67475263),
 (67475263, 67416985),
 (67416985, 67377060),
 (67377060, 67448114),
 (67448114, 5425641239),
 (5425641239, 67407188),
 (67407188, 67479372),
 (67479372, 67479285),
 (67479285, 67413259),
 (67413259, 5444597516),
 (5444597516, 7297946661),
 (7297946661, 5506569014),
 (5506569014, 67376113),
 (67376113, 6988472159),
 (6

In [66]:
df_edges.columns

Index(['source', 'target', 'reverse_link', 'azimuth', 'linkid', 'osmid',
       'link_type', 'name', 'highway', 'bridge', 'tunnel', 'bl', 'pbl', 'mu',
       'speedlimit_range_mph', 'lanes_per_direction', 'up_grade', 'length_ft',
       'vehicle_separation'],
      dtype='object')

In [21]:
chosen_links = links.loc[path]

summary_attributes = {}

#specify columns to summarize
bool_cols = ['bl','mu','pbl']
cols = ['link_type','highway','functional_class','speedlimit_range_mph','lanes_per_direction']


In [22]:
for bool_col in bool_cols:
    total_length = chosen_links[chosen_links[bool_col] == 1].length.sum()
    summary_attributes[bool_col] = np.round(total_length/chosen_links.length.sum(),2)

for col in cols:
    for unique_val in chosen_links[col].unique():
        if (unique_val != None) | (unique_val == np.nan):
            total_length = chosen_links[chosen_links[col] == unique_val].length.sum()
        else:
            continue
        if isinstance(unique_val,str) == False:
            unique_val = str(unique_val)
        summary_attributes[col+'.'+unique_val] = np.round(total_length/chosen_links.length.sum(),2)

summary_attributes


{'bl': 0.44,
 'mu': 0.13,
 'pbl': 0.0,
 'link_type.bike': 0.13,
 'link_type.road': 0.87,
 'highway.footway': 0.07,
 'highway.tertiary': 0.22,
 'highway.residential': 0.63,
 'highway.cycleway': 0.06,
 'highway.secondary': 0.02,
 'functional_class.local': 0.85,
 'functional_class.collectors/minor arterials': 0.02,
 'speedlimit_range_mph.21-30 MPH': 0.87,
 'lanes_per_direction.1': 0.85,
 'lanes_per_direction.2-3': 0.02}

# Turn Attributes

In [23]:
#TODO recalculate bearing
#create pseudo graph for modeling turns
df_edges, pseudo_links, pseudo_G = modeling_turns.create_pseudo_dual_graph(links,'A','B','linkid','oneway')


In [40]:
source = list(pseudo_G.nodes())[0]
target = list(pseudo_G.nodes())[420]

In [45]:
source = (68209677, 68209675)
target = (69200243, 69465418)

import networkx as nx
length, path = nx.single_source_dijkstra(pseudo_G,source,target)

In [24]:
edge_list = [(linkids[i],linkids[i+1]) for i in range(len(linkids)-1)]
edge_list

[(32641, 33174),
 (33174, 33175),
 (33175, 35233),
 (35233, 35234),
 (35234, 35235),
 (35235, 32803),
 (32803, 36922),
 (36922, 36923),
 (36923, 32827),
 (32827, 36925),
 (36925, 32826),
 (32826, 33089),
 (33089, 33090),
 (33090, 32834),
 (32834, 32721),
 (32721, 32598),
 (32598, 33239),
 (33239, 33240),
 (33240, 33241),
 (33241, 32728),
 (32728, 32727),
 (32727, 32599),
 (32599, 35293),
 (35293, 35295),
 (35295, 32479),
 (32479, 32480),
 (32480, 32482),
 (32482, 32739),
 (32739, 32736),
 (32736, 32613),
 (32613, 32614),
 (32614, 32612),
 (32612, 32611),
 (32611, 32610),
 (32610, 32747),
 (32747, 32748),
 (32748, 32749),
 (32749, 33266),
 (33266, 33273),
 (33273, 33274),
 (33274, 33275),
 (33275, 33276),
 (33276, 33277)]

In [50]:
edge_list = [(*path[i],*path[i+1]) for i in range(len(path)-1)]
edge_list

[(68209677, 68209675, 68209675, 3489068402),
 (68209675, 3489068402, 3489068402, 68326014),
 (3489068402, 68326014, 68326014, 6705199299),
 (68326014, 6705199299, 6705199299, 1821421030),
 (6705199299, 1821421030, 1821421030, 7070716830),
 (1821421030, 7070716830, 7070716830, 7070716829),
 (7070716830, 7070716829, 7070716829, 68266456),
 (7070716829, 68266456, 68266456, 68326000),
 (68266456, 68326000, 68326000, 68171958),
 (68326000, 68171958, 68171958, 655203025),
 (68171958, 655203025, 655203025, 68143915),
 (655203025, 68143915, 68143915, 68325990),
 (68143915, 68325990, 68325990, 68325987),
 (68325990, 68325987, 68325987, 68325985),
 (68325987, 68325985, 68325985, 68307446),
 (68325985, 68307446, 68307446, 68165369),
 (68307446, 68165369, 68165369, 68389876),
 (68165369, 68389876, 68389876, 384328862),
 (68389876, 384328862, 384328862, 68168122),
 (384328862, 68168122, 68168122, 68172175),
 (68168122, 68172175, 68172175, 68172159),
 (68172175, 68172159, 68172159, 68332877),
 (6817

In [48]:
pseudo_links.columns

Index(['source_A', 'source_B', 'target_A', 'target_B', 'source_azimuth',
       'source_reverse_link', 'source_linkid', 'target_azimuth',
       'target_reverse_link', 'target_linkid', 'azimuth_change', 'turn_type',
       'source', 'target'],
      dtype='object')

In [53]:
pseudo_links.set_index(['source_A','source_B','target_A','target_B']).loc[edge_list,'turn_type'].value_counts()

backwards    27
straight     23
left         13
right         9
Name: turn_type, dtype: int64